# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest as pz

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<b> 1. What test is appropriate for this problem? </b>

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id                    4870 non-null object
ad                    4870 non-null object
education             4870 non-null int8
ofjobs                4870 non-null int8
yearsexp              4870 non-null int8
honors                4870 non-null int8
volunteer             4870 non-null int8
military              4870 non-null int8
empholes              4870 non-null int8
occupspecific         4870 non-null int16
occupbroad            4870 non-null int8
workinschool          4870 non-null int8
email                 4870 non-null int8
computerskills        4870 non-null int8
specialskills         4870 non-null int8
firstname             4870 non-null object
sex                   4870 non-null object
race                  4870 non-null object
h                     4870 non-null float32
l                     4870 non-null float32
call                  4870 non-null float32
city        

-  CLT applies since the observations are independent and the sample size is sufficiently large to assume a normal distribution.
 
-  A z-test of proportions can be used. A Chi-squared test can also be used.

##  2. What are the null and alternate hypotheses?

- The null hypothesis is that probability of a callback for 'b' is equal to 'w' or p_b = p_w.

- The alternate hypothesis is p_b≠p_w 

## 3. Compute margin of error, confidence interval, and p-value.

In [6]:
w = data[data.race=='w']
b = data[data.race=='b']

In [7]:
def ztest_proportion_two_samples(x1,n1,x2,n2,one_sided=False):
    p1=x1/n1
    p2=x2/n2
    p=(x1+x2)/(n1+n2)
    se=np.sqrt(p*(1-p)*(1/n1+1/n2))
    
    z=(p1-p2)/se
    p=1-stats.norm.cdf(abs(z))
    p*=2-one_sided
    return z,p

In [8]:
w_call=np.array(w.iloc[:,20])
b_call=np.array(b.iloc[:,20])

w_x1=np.sum(w_call)
b_x1=np.sum(b_call)

w_n1=len(w_call)
b_n1=len(b_call)

z,p=ztest_proportion_two_samples(w_x1,w_n1,b_x1,b_n1,False)

In [9]:
def compute_SE_two_sample(x1,n1,x2,n2):
    p1=x1/n1
    p2=x2/n2
    se=p1*(1-p1)/n1+p2*(1-p2)/n2
    return np.sqrt(se)

se=compute_SE_two_sample(w_x1,w_n1,b_x1,b_n1)

In [10]:
def z_conf_int_two_samples(x1,n1,x2,n2,confidence=0.95):
    p1=x1/n1
    p2=x2/n2
    se=compute_SE_two_sample(x1,n1,x2,n2)
    z_critical=stats.norm.ppf((1+confidence)/2)
    return p1-p2-z_critical*se,p1-p2+z_critical*se

low,up=z_conf_int_two_samples(w_x1,w_n1,b_x1,b_n1,confidence=0.95)
print(low,up)    


0.016777728181230755 0.04728798023766041


In [11]:
margin_error=se*z
print("z value is "+str(z))
print("p value is "+str(p))
print("confidence interval is "+ str(low)+' '+str(up))
print("margin of error is "+str(margin_error))


z value is 4.108412152434346
p value is 3.983886837577444e-05
confidence interval is 0.016777728181230755 0.04728798023766041
margin of error is 0.031977294305202825


## 4. Write a story describing the statistical significance in the context or the original problem.
### Summary 
-  The original problem asks whether race has a significant impact on the rate of callbacks for resumes. It is understood clearly that a black sounding name negatively influences if the person receives a callback or not.

## 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?¶
-  No, it does not mean that race/name is the most important factor It means is race/name plays a role which is statistically significant. We need more data to determine if/what other factors may be influencing the outcome.